In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import nltk
from nltk.tokenize import word_tokenize

In [2]:
#Read in data from CSV files.

features = pd.read_csv('clean_data/features_encoded.csv')

cost_codes = pd.read_csv('clean_data/cost_code_encoded.csv')

In [3]:
#Use sklearn train test split to split the data into training and testing sets. 
#Testing set is 20% of total dataset size.
#Stratify the data so we don't introduce bias in the sets.

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    cost_codes,
                                                    test_size = 0.2,
                                                    stratify = cost_codes
                                                   )


In [5]:
features['Description'] = features['Description'].apply(word_tokenize)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer)

,Description,Units,Unit Cost,Cost,Vendor_1110438 B.C. Ltd,Vendor_1ST-R8 Welding & Fabrication Ltd.,Vendor_4s Scaffolding,Vendor_596143 BC LTD DBA AVANTE 2000,Vendor_7 Star Security Services Inc,Vendor_A & A Testing Ltd.,...,Unit of Measure_kg,Unit of Measure_km,Unit of Measure_kw,Unit of Measure_l,Unit of Measure_m,Unit of Measure_m2,Unit of Measure_m3,Unit of Measure_m3,Unit of Measure_mL,Unit of Measure_t
0,"[railing, per, quote, aug, 13, 2015]",0.000000,0.000000,0.218551,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[metal, ramp]",0.000000,0.000000,0.347495,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[painting, two, office]",0.000000,0.000000,0.393391,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[demolition, wall, carpeting, platform]",0.000000,0.000000,0.328700,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[hollow, metal, frame]",0.062893,0.876712,0.139872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
